<h1 style="text-align: center;">
    ATSC 409 Project: 1D Radiative Model
</h1>
<h5 style="text-align: center;">
    Anirudh Duggal, Clinton Macadam, and Hanrui Duan
</h5>
<!-- # ATSC 409 Project: 1D Radiative Model
##### Anirudh Duggal, Clinton Macadam, and Hanrui Duan -->

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# Runge-Kutta 4th order
def rk4(f, x0, y0, x1, n):
    h = (x1 - x0) / n
    x = np.arange(x0, x1, h)
    y = np.zeros((n + 1))
    y[0] = y0
    for i in range(n):
        k1 = h * f(x[i], y[i])
        k2 = h * f(x[i] + h / 2, y[i] + k1 / 2)
        k3 = h * f(x[i] + h / 2, y[i] + k2 / 2)
        k4 = h * f(x[i] + h, y[i] + k3)
        y[i + 1] = y[i] + (k1 + 2 * k2 + 2 * k3 + k4) / 6
    return x, y

<h4 style="text-align: center;">
    Constants
</h4>

In [17]:
Ce = 2.08e-5  # J/m^2 * C
F = 3.80      # W/(m^2 * C) 
A = 204       # W m^-2 
B = 2.17      # W m^-2 C^-1

2.08